# Introduction to the NCAS CF Data Tools, cf-python and cf-plot

***

## Setting up

**In this section we set up this Notebook, import the libraries and check the data we will work with, ready to use the libraries within this notebook.**

Run some set up for nice outputs in this Jupyter Notebook (not required in interactive Python or a script):

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

Import cf-python and cf-plot:

In [ ]:
import cfplot as cfp
import cf

Inspect the versions of cf-python and cf-plot and the version of the CF Conventions those are matched to:

In [ ]:
print("cf-python version is:", cf.__version__)
print("cf-plot version is:", cfp.__version__)
print("CF Conventions version is:", cf.CF())

<div class="alert alert-block alert-info">
<i>Note:</i> you can work with data compliant by any other version of the CF Conventions, or without (much) compliance, but the CF Conventions version gives the maximum version that these versions of the tools understand the features of.
</div>

Finally, see what datasets we have to explore:

<div class="alert alert-block alert-info">
<i>Note:</i> in a Jupyter Notebook, '!' preceeeds a shell command, so this is a terminal command and not Python
</div>

In [ ]:
!ls ../ncas_data

***

## 1. Reading dataset(s) and viewing the (meta)data at different detail levels

**In this section we look at a basic use of cf-python, reading in one or more datasets from file and inspecting the data and the metadata at different levels of detail to suit the amount of information you want to see.**

<div class="alert alert-block alert-info">
<i>Note:</i> In cf-python and when discussing related code and datasets, we use terminology from the CF Data Model (for more detail see: <a href="url">https://ncas-cms.github.io/cf-python/cf_data_model.html</a>). For example cf-python methods are named in relation to concepts from this data model. We don't have time to cover this in detail but for this session it is useful to know the following terms:

<ul>
    <li><b>field</b>: a self-contained cf-python object corresponding to a netCDF data variable with all of its (CF) metadata attached;</li>
    <li><b>field list</b>: a list of lields (see above), stored as its own cf-python object 'FieldList' which is similar to a Python list;</li>
    <li><b>coordinate</b>: a (CF) metadata concept which corresponds to netCDF coordinate variables. One or more coordinates are defined on every field as either 'dimension' or 'auxiliary' coordinate objects in cf-python.</li>
</ul>
</div>

The examples from this section should help you to familiarise yourself with these terms and their practical usage.

### a) Reading in data and extracting the _field_ of interest

Read a chosen data file. Sometimes datasets have descriptive names but this one doesn't, so let's find out what it is!

In [ ]:
fieldlist = cf.read("../ncas_data/data1.nc")

See the 'fieldlist' that cf-python interprets from the data read in:

In [ ]:
fieldlist

Select a particular field from the fieldlist of interest:

In [ ]:
field = fieldlist[0]

### b) Inspecting the _field_ of interest with different amounts of detail

View the field with **minimal detail**, i.e. a one-line summary:

In [ ]:
field

Or you can view it with a **medium level of detail** with the Python built-in `print` function:

In [ ]:
print(field)

A final option is to view it with **maximal detail** using the `dump()` method:

In [ ]:
field.dump()

### c) Inspecting a metadata _construct_ e.g. _coordinate_ from the _field_ of interest

Use the same approach to view a particular metadata aspect, for example the latitude coordinate:

In [ ]:
lat = field.coordinate("latitude")

In [ ]:
lat

In [ ]:
print(lat)

In [ ]:
lat.dump()

### d) Inspecting a data array of interest

Likewise, the same approach works to view the data itself in the field (i.e. the underlying arrays). First, grab the data from the field with the `data` attribute:

In [ ]:
data = field.data

Then view it in a chosen level of detail as with the above objects:

In [ ]:
data

In [ ]:
print(data)

In [ ]:
data.dump()

If you want to see more of the data array itself, you can access it with the `array` attribute. Beware, for real-life datasets:

* this will be large and Python will likely truncate it so your screen isn't spammed with sub-arrays of values!
* it is computationally intensive to access the underlying data array if it is large, especially if it is multi-dimensional, so your computer will often have to work hard to get the array, so use the `array` method sparingly (only when needed)!

In [ ]:
data.array

***

## Where to find more information and resources on the NCAS CF Data Tools

Here are some links relating to the NCAS CF Data Tools and this training.

* This training, with further material, is hosted online and there are instructions for setting up the environment so you can work through it in your own time: https://github.com/NCAS-CMS/cf-tools-training.
* The cf-python documentation lives at https://ncas-cms.github.io/cf-python/.
* The cf-python code lives on GitHub at https://github.com/NCAS-CMS/cf-python. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-python/issues.
* The cf-plot documentation lives at https://ncas-cms.github.io/cf-plot/build/.
* The cf-plot code lives on GitHub at https://github.com/NCAS-CMS/cf-plot. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-plot/issues.
* There is a technical presentation about the NCAS CF Data Tools avaialble from https://hps.vi4io.org/_media/events/2020/summer-school-cfnetcdf.pdf.
* The website of the CF Conventions can be found at https://cfconventions.org/.
* The landing page for training into the CF Conventions is found here within the website above: https://cfconventions.org/Training/.

If you have any queries after this course, please either use the Issue Trackers linked above or you can email me at: sadie.bartholomew@ncas.ac.uk.

***